# Trend Fitness

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer

In [13]:
class RecommendationSystem:
    def __init__(self, df):
        self.df = df

    def collaborative_filtering(self, user_id, num_recommendations=5):
        user_profile = self.df.loc[self.df['User ID'] == user_id]

        # Extract relevant features for collaborative filtering
        user_features = user_profile[['Primary Fitness Goal', 'Physical Activity Level']]
        all_features = self.df[['Primary Fitness Goal', 'Physical Activity Level']]

        # Concatenate user features with all features
        concatenated_features = pd.concat([user_features, all_features])

        # Encode categorical variables
        encoder = OneHotEncoder(drop='first', handle_unknown='ignore')
        encoded_features = encoder.fit_transform(concatenated_features)

        #print("Categories discovered by the encoder:")
        #print(encoder.categories_)

        # Split encoded features back into user and all features
        user_features_encoded = encoded_features[:1]
        all_features_encoded = encoded_features[1:]

        #print("\nShapes of encoded user and all features:")
        #print(user_features_encoded.shape, all_features_encoded.shape)

        # Compute similarity between the target user and all users
        similarities = cosine_similarity(user_features_encoded, all_features_encoded)[0]

        #print("\nSimilarities array shape:")
        #print(similarities.shape)

        # Find most similar users
        similar_users_indices = similarities.argsort()[::-1][1:]  # Exclude the target user
        similar_users = self.df.iloc[similar_users_indices]

        # Generate recommendations based on similar users' preferences
        recommendations = {
            'Nutritional Preferences': similar_users['Nutritional Preferences'].tolist()[:num_recommendations],
            'Fitness Environment': similar_users['Fitness Environment'].tolist()[:num_recommendations]
        }
        return recommendations
        
   
    def content_based_filtering(self, user_id, num_recommendations=5):
        user_profile = self.df.loc[self.df['User ID'] == user_id]

        # Extract user preferences
        user_preferences = user_profile[['Primary Fitness Goal', 'Physical Activity Level', 'Nutritional Preferences', 'Fitness Environment', 'Time Commitment']]

        # Extract item features
        item_features = self.df[['Primary Fitness Goal', 'Physical Activity Level', 'Nutritional Preferences', 'Fitness Environment', 'Time Commitment']]

        # Encode user preferences
        mlb = MultiLabelBinarizer()
        user_features_encoded = mlb.fit_transform(user_preferences.values)

        #print("User preferences shape:", user_features_encoded.shape)

        # Encode item features
        item_features_encoded = mlb.transform(item_features.values)

        #print("Item features shape:", item_features_encoded.shape)

        # Compute similarity between items and user preferences
        similarities = cosine_similarity(user_features_encoded, item_features_encoded)

        #print("Similarities array shape:", similarities.shape)

        # Sort items based on similarity and recommend top ones
        similar_items_indices = similarities.argsort()[0][::-1]
        recommendations = {
            'Fitness Environment': self.df.iloc[similar_items_indices]['Fitness Environment'].tolist()[:num_recommendations],
            'Nutritional Preferences': self.df.iloc[similar_items_indices]['Nutritional Preferences'].tolist()[:num_recommendations],
        }

        return recommendations

In [14]:

# Read dataset
data = pd.read_csv('fitness_dataset.csv')

df = pd.DataFrame(data)


# Instantiate recommendation system
recommendation_system = RecommendationSystem(df)

# Example usage
user_id = 200

collaborative_recommendations = recommendation_system.collaborative_filtering(user_id)
content_based_recommendations = recommendation_system.content_based_filtering(user_id)

print("Collaborative Filtering Recommendations:", collaborative_recommendations)
print("Content-Based Filtering Recommendations:", content_based_recommendations)

Collaborative Filtering Recommendations: {'Nutritional Preferences': ['Low Carb Diet', 'Low Calorie Diet', 'Low Carb Diet', 'Keto Diet', 'Keto Diet'], 'Fitness Environment': ['Outdoor Bootcamp', 'HIIT Studio', 'Outdoor Bootcamp', 'Boxing Gym', 'Boxing Gym']}
Content-Based Filtering Recommendations: {'Fitness Environment': ['Outdoor Bootcamp', 'Outdoor Bootcamp', 'Outdoor Bootcamp', 'Outdoor Bootcamp', 'Outdoor Bootcamp'], 'Nutritional Preferences': ['Low Carb Diet', 'Low Carb Diet', 'Low Carb Diet', 'Low Carb Diet', 'Low Carb Diet']}


d:\workspace\Django Project\venv\Lib\site-packages\sklearn\preprocessing\_label.py:900: UserWarning: unknown class(es) ['1-3 hours per week', '5-7 hours per week', 'Alkaline Diet', 'Anti-Inflammatory Diet', 'Athletic Performance', 'Athletic Training Facility', 'Balanced Diet', 'Beach Yoga', 'Bodybuilding Gym', 'Bodyweight Fitness Park', 'Boxing Gym', 'Calcium-Rich Diet', 'Commercial Gym', 'Community Center', 'Community Fitness Center', 'Community Recreation Center', 'Core Strength', 'CrossFit Box', 'Cycling Club', 'Cycling Studio', 'Endurance', 'Fitness Center', 'Flexibility', 'Gluten-Free Diet', 'Group Fitness Classes', 'Gym', 'HIIT Studio', 'Heart-Healthy Diet', 'High', 'High Carb Diet', 'High Fiber Diet', 'High Protein Diet', 'Home Gym', 'Home Workout Space', 'Home Workouts', 'Indoor Cycling Studio', 'Injury Prevention', 'Intermittent Fasting', 'Keto Diet', 'Ketogenic Diet', 'Low', 'Low Calorie Diet', 'Low Fat Diet', 'Low Glycemic Diet', 'Low Sodium Diet', 'Macrobiotic Diet', 'Medit